In [1]:
import numpy as np
import laspy as lp
import pptk
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn import metrics
from sklearn.cluster import DBSCAN
from scipy import stats
import scipy
import seaborn as sns
from sklearn.mixture import GaussianMixture

In [2]:
def diagnostic_plots(variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(variable, bins=100)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(variable, dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=variable)
    plt.title('Boxplot')

    plt.show()

In [3]:
def get_ideal_num_clusters(X):
    bic_scores = []
    for i in range(1, 6):
        gm = GaussianMixture(n_components=i, random_state=0).fit(np.expand_dims(X, axis=-1))
        bic = gm.score(np.expand_dims(X, axis=-1))
        bic_scores.append(bic)
        print(f'bic score is {bic} for {i} clusters')
    return np.argmin(bic_scores) + 1     

In [4]:
def intensity_diff(i1, i2):
    return abs(i1-i2)

def cluster(list_intensities, epsilon, min_samples):
    dist_matrix = scipy.spatial.distance.pdist(list_intensities, metric=intensity_diff).astype(np.float16)
    dist_matrix = scipy.spatial.distance.squareform(dist_matrix)
    db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='auto', metric='precomputed').fit(dist_matrix)
    cluster_labels = db.labels_
    return cluster_labels

In [5]:
def make_df(point_cloud):
    arr = point_cloud.points.array
    columns = list(dict(arr.dtype.fields).keys())
    l = list((map(lambda x: list(x), arr[::10])))
    df = pd.DataFrame(l, columns =columns)
    df.to_csv('records.csv', index=False)

In [6]:
def visualize_pointcloud(param):
    points, intensities, labels = read_df()
    if param == 'intensity':
        intensities = np.clip(intensities, a_min = 1, a_max=255)
        max_intensity = max(intensities)
        colors = plt.get_cmap("tab20")(intensities / (max_intensity if max_intensity > 0 else 1))[:,:-1]
    else:
        max_label = max(labels)
        colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))[:,:-1]
    v = pptk.viewer(points)
    v.attributes(colors)

In [16]:
def read_df():
    df = pd.read_csv('data/records.csv', usecols=['X', 'Y', 'Z', 'intensity', 'raw_classification'])
    arr = df.to_numpy()
    points = arr[:, :3]/1000
    intensities = arr[:, 3]
    labels = arr[:,4]
    return points, intensities, labels

In [19]:
def read_df_ground_points():
    df = pd.read_csv('data/records.csv', usecols=['X', 'Y', 'Z', 'intensity', 'raw_classification'])
    pcd = o3d.io.read_point_cloud('')
    arr = df.to_numpy()[::10]
    labels = arr[:, -1]
    points = arr[:, :3] / 1000
    intensities = arr[:, 3]
    points = points[labels == 2]
    intensities = intensities[labels == 2]
    pcd.points.extend(points)
    inten_labels = GaussianMixture(n_components=3, random_state=0).fit_predict(np.expand_dims(intensities, axis=-1))
    return pcd, inten_labels, labels

In [8]:
def plot_hist(arr):
    # todo add more functionality   
    plt.hist(arr[:,-1], bins=20)
    pass

In [9]:
def get_factored_points(points, factor):
    return points[::factor]

In [10]:
def write_las_file(file_path, points):
    header = lp.header.LasHeader()
    las_data = lp.lasdata.LasData(header)
    las_data.x = points[:,0]
    las_data.y = points[:,1]
    las_data.Z = points[:,2]
    las_data.write(file_path)

In [11]:
def group_into_grids(points):
    pt_cloud_grids = []
    sorted_by_x = points[np.argsort(points[:,0])]
    splits_along_x = np.array_split(sorted_by_x, 10)
    for split in splits_along_x:
        sorted_by_y = split[np.argsort(split[:,1])]
        splits_along_y = np.array_split(sorted_by_y, 10)
        pt_cloud_grids.extend(splits_along_y)
    return pt_cloud_grids  

In [12]:
def visualize_classes(class_val):
    '''
    class val can be one of:
    {1, 2, 6, 9, 26}
    '''
    points_o, intensities_o, labels_o = read_df()
    points = points_o[labels_o==class_val]
    labels = labels_o[labels_o==class_val]
    intensities = intensities_o[labels_o==class_val]
    max_label = max(labels)
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))[:,:-1]
    v = pptk.viewer(points)
    v.attributes(colors)
    return points, labels, intensities

In [17]:
def segment_trees_and_buildings():
    """
    Segments non ground points by doing a segregation based on intensity
    using Gaussing mixture model.
    Assumption: We mainly have trees and buildings as non ground objects
    and their LiDAR intensities come from 2 different distributions.
    :return: None
    """
    points_o, intensities_o, labels_o = read_df()
    heights = points_o[:,2]
    points = points_o[heights > 8]
    intensities = intensities_o[heights > 8]
    labels = GaussianMixture(n_components=2, random_state=0).fit_predict(np.expand_dims(intensities, axis=-1))
    max_label = max(labels)
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))[:, :-1]
    v = pptk.viewer(points)
    v.attributes(colors)

In [23]:
def fit_multiple_horizontal_planes(num_planes):
    """
    colorize the point cloud accross multiple horizontal planes
    thru it
    :param num_planes: number of planes or horizontal sections
    :return:
    """
    points_o, intensities_o, labels_o = read_df()
    heights = points_o[:, -1]
    intervals = np.linspace(int(np.min(heights)), int(np.max(heights)), num_planes)
    color_labels = np.zeros((len(points_o),))
    for i in range(len(intervals) - 1):
        mask = np.bitwise_and((intervals[i] < heights), (heights < intervals[i + 1]))
        args_filtered = np.argwhere(mask)
        color_labels[args_filtered] = i
    max_label = np.max(color_labels)
    colors = plt.get_cmap("tab20")(color_labels / (max_label if max_label > 0 else 1))[:, :-1]
    v = pptk.viewer(points_o)
    v.attributes(colors)

### Segmentation of non-ground entities i.e. buildings and Trees

In [18]:
segment_trees_and_buildings()

### RANSAC based plain fitting thru ground points

In [22]:
# This takes a little time :)
import ransac_seg
ransac_seg.main()

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for  (format: auto).
pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.
RANSAC_loop_visualization


### Horizontal plain (z = k) fittinng and colorization

In [24]:
fit_multiple_horizontal_planes(num_planes=15)